# Bar Element Example

Based on [Schuster Engineering](https://www.youtube.com/watch?v=bm7nmaJmxQs&list=PLBwQ5Llf4Q_Vj8xKcxsbzIfdqrtzcHLbe&index=14).

<img src="assets/images/bar_example1.png" width="400"/>

In [1]:
import numpy as np
from numpy.linalg import inv
import pandas as pd
import matplotlib.pyplot as plt
import scipy
from scipy.integrate import quad
import sys
sys.path.insert(1, 'libraries')
import utils

## Input data
Define the input data.

In [2]:
materials=[['mat1',30E+6]]
sections=[['section 1',2,11]]
nodes=[['node 1', 0, 0],['node 2', 30, 0],['node 3', 60, 0]]
elements=[['element 1','node 1','node 2','section 1','mat1','bar'],['element 2','node 2','node 3','section 1','mat1','bar']]
restraints=['node 1']

<p style="color:red;">DO NOT MODIFY THE CODE BELOW</p>

In [3]:
materials=pd.DataFrame(materials,columns=['name','E'])
sections=pd.DataFrame(sections,columns=['name','A','I'])
nodes=pd.DataFrame(nodes,columns=['name','x','y'])
elements=pd.DataFrame(elements,columns=['name','n1','n2','section','material','type'])

In [4]:
f_b=[['element 1',0,300],['element 2',300,600]]
f_b=pd.DataFrame(f_b,columns=['element','end 1','end 2'])

In [5]:
# add element lengths based on point coordinates
L=[]
for i in range(len(elements)):
    if elements['type'][i] == 'bar':
        idx1=utils.df_index(nodes,elements['n1'][i],'name')
        idx2=utils.df_index(nodes,elements['n2'][i],'name')
        x1=nodes['x'][idx1]
        y1=nodes['y'][idx1]
        x2=nodes['x'][idx2]
        y2=nodes['y'][idx2]
        L_temp=((x2-x1)**2+(y2-y1)**2)**0.5
        L.append(L_temp)
elements.insert(6, "length", L, True)

In [6]:
def N_bar(x,L): return np.array([1-x/L,x/L])

def f_bar(x,e1,e2,L): return (e2-e1)/L*x+e1

def integMult(e1,e2,L):
    mat = []
    for i in range(2):
        mat.append(quad(lambda x: N_bar(x,L)[i]*f_bar(x,e1,e2,L), 0, L)[0])
    return mat

def k_bar(A,E,L):
    return A*E/L*np.array([[1,-1],[-1,1]])

In [7]:
f_calc=[]
for i in range(len(f_b['element'])):
#     e1=utils.df_value(f_b,f_b['element'][i],'element','end 1')
#     e2=utils.df_value(f_b,f_b['element'][i],'element','end 2')
    e1=f_b['end 1'][i]
    e2=f_b['end 2'][i]
    L=utils.df_value(elements,f_b['element'][i],'name','length')
    f_calc.append(integMult(e1,e2,L))
f_calc=pd.DataFrame(f_calc,columns=['calc 1','calc 2'])
f_b=f_b.join(f_calc)

In [8]:
# Assemble global force vector
f_glob = np.zeros((len(nodes),1))
for i in range(len(f_b)):
    if elements['type'][i] == 'bar':
        n1=utils.df_value(elements,f_b['element'][i],'name','n1')
        idx1=utils.df_index(nodes,n1,'name')
        f_glob[idx1]=f_glob[idx1]+f_b['calc 1'][i]
        n2=utils.df_value(elements,f_b['element'][i],'name','n2')
        idx2=utils.df_index(nodes,n2,'name')
        f_glob[idx2]=f_glob[idx2]+f_b['calc 2'][i]

In [9]:
# Assemble element and global stiffness matrices
k_glob = np.zeros((len(nodes),len(nodes)))
k_elms =[]
for i in range(len(elements)):
    if elements['type'][i] == 'bar':
        A = utils.df_value(sections,elements['section'][i],'name','A')
        E = utils.df_value(materials,elements['material'][i],'name','E')
        idx1=utils.df_index(nodes,elements['n1'][i],'name')
        idx2=utils.df_index(nodes,elements['n2'][i],'name')
        L=elements['length'][i]
        k_elms.append(k_bar(A,E,L))
        k_glob[idx1][idx1]=k_glob[idx1][idx1]+k_elms[i][0][0]
        k_glob[idx1][idx2]=k_glob[idx1][idx2]+k_elms[i][0][1]
        k_glob[idx2][idx1]=k_glob[idx2][idx1]+k_elms[i][1][0]
        k_glob[idx2][idx2]=k_glob[idx2][idx2]+k_elms[i][1][1]

In [10]:
rest_vect=np.zeros((len(nodes),1))
k_glob_red=k_glob
f_glob_red=f_glob
indices=[]
for i in range(len(restraints)):
    rest_vect[utils.df_index(nodes,restraints[i],'name')]=1
    indices.append(i)
k_glob_red = np.delete(k_glob_red,indices,0)
k_glob_red = np.delete(k_glob_red,indices,1)
f_glob_red = np.delete(f_glob,indices,0)

In [21]:
# displacements
displacements=np.matmul(inv(k_glob_red),f_glob_red)
displacements

array([[0.00825],
       [0.012  ]])

In [20]:
# reaction forces
reactions=[]
for i in range(len(restraints)):
    reactions.append(k_glob[i])

array([[0.00825],
       [0.012  ]])

In [22]:
np.matmul(k_glob[0],0

array([ 2000000., -2000000.,        0.])

In [13]:
# e1=utils.df_value(f_b,'element 1','element','end 1')
# e2=utils.df_value(f_b,'element 1','element','end 2')
# L=30
# #result = quad(lambda x: N_bar(x,L)[1]*f_bar(x,e1,e2,L), 0, L)[0]
# result=integMult(e1,e2,L)
# print("the result is", result)

In [14]:
from scipy.integrate import quad
def f(x,a1,a2,L):  #a is a parameter, x is the variable I want to integrate over
    return a1+x*(a2-a1)/L
I = quad(f, 0, 1, args=(5,10,30))
I

(5.083333333333333, 5.643633708511212e-14)

In [15]:
import numpy
from scipy.integrate import trapz
g=lambda x: numpy.array([[x,2*x,3*x],[x**2,x**3,x**4]])
xv=numpy.linspace(0,100,200)
print(trapz(g(xv)))

[[9.95000000e+03 1.99000000e+04 2.98500000e+04]
 [6.63341709e+05 4.97512563e+07 3.98016750e+09]]


In [16]:
from math import sin, cos, pi
from scipy.integrate import quad
from numpy import vectorize
a = [sin, cos]
vectorize(quad)(a, 0, pi)
#(array([  2.00000000e+00,   4.92255263e-17]), array([  2.22044605e-14,   2.21022394e-14]))

(array([2.00000000e+00, 3.67759339e-17]),
 array([2.22044605e-14, 2.21022394e-14]))